In [1]:
import torch.nn.functional as F
import torch.nn as nn
import torch
import lightning.pytorch as pl
import torchmetrics

import pandas as pd
import numpy as np

from IPython.display import HTML, display
import os
from types import SimpleNamespace

from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification.accuracy import BinaryAccuracy
import lightning as L
import matplotlib
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import numpy as np
import seaborn as sns
import tabulate
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
%matplotlib inline



from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.lr_monitor import LearningRateMonitor
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from wavelets_cnn import ResNet

/mnt/sda5/conda/envs/open-mmlab/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
torch.cuda.empty_cache()

In [3]:
matplotlib_inline.backend_inline.set_matplotlib_formats(
    "svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()


RANDOM_STATE = 42
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/ConvNets")


# Function for setting the seed
L.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")



Global seed set to 42


Extracted beats, as explained in
Section III-A, are used as inputs. Here, all convolution layers
are applying 1-D convolution through time and each have 32
kernels of size 5. We also use max pooling of size 6 and stride
2 in all pooling layers. The predictor network consists of five
residual blocks followed by two fully-connected layers with
32 neurons each and a softmax layer to predict output class probabilities. Each residual block contains two convolutional
layers, two ReLU nonlinearities [19], a residual skip connec-
tion [20], and a pooling layer. In total, the resulting network
is a deep network consisting of 13 weight layers.

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)

    def forward(self , x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = out + residual  # Residual connection
        out = self.relu2(out)
        out = self.pool(out)
        return out

class Baseline(nn.Module):
    def __init__(self, sequence_len, n_classes, n_blocks=5):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=12, out_channels=32, kernel_size=5,stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(32)
        self.residual_blocks = nn.Sequential(
            ResidualBlock(32),
            ResidualBlock(32),
            ResidualBlock(32),
            ResidualBlock(32),
            ResidualBlock(32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(sequence_len, 32), # 20
            nn.ReLU(),
            nn.Linear(32, n_classes),
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.residual_blocks(out)
        out = out.view(out.size(0), -1)  # Flatten to [batch_size, channels * sequence_length]

        return self.classifier(out)
        
 

In [5]:
test_beat = np.load('./transformed_train/00034_hr.npy')
ecg_lead = 9
segment_num = 1
print(test_beat.shape)
model = ResNet(num_classes=1)
#model = Baseline(test_beat.shape[1]-128, 1)


test_beat = test_beat.reshape((1,12,375, -1))
test_y = torch.tensor([[1.]])
criterion = nn.BCEWithLogitsLoss()
print("test beat shape", test_beat.shape)
res= model(torch.from_numpy(test_beat).float())
print(res.shape)
criterion(res, test_y)

(12, 375, 375)
test beat shape (1, 12, 375, 375)
torch.Size([1, 1])


tensor(1.2583, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [6]:
import os
import pandas as pd
from torch.utils.data import Dataset

class DatasetECG(Dataset):
    def __init__(self, annotations_file, signals_dir):
        """
        annotantions_file - path to the annotations dataframe. 
                            First column should be name of the record, second - strat_fold then labels 
        
        signals_dir - path to the directory with transformed signals
        """
        self.signals_labels = pd.read_csv(annotations_file)
        self.signals_dir = signals_dir 
        self.ecg_lead = 9
        self.segment_num = 1

    def __len__(self):
        return len(self.signals_labels)

    def __getitem__(self, idx):
        signals_path = os.path.join(self.signals_dir, self.signals_labels.iloc[idx, 0]+ ".npy")
        signal = np.load(signals_path).astype(np.float32)        

        # iloc[idx, 2:] 2 is because first column is a record name
        labels = torch.from_numpy(self.signals_labels.iloc[idx, 2:].values.astype(int)).float()
        return signal, labels


In [7]:
train_dataset = DatasetECG("./train_annotations.csv", "transformed_train")
val_dataset = DatasetECG("./val_annotations.csv", "transformed_train")

In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

In [9]:
class LitBaseline(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion = nn.BCEWithLogitsLoss()
        self.train_score = F1Score()
        self.accuracy = BinaryAccuracy()
        self.val_score = F1Score()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0),12, -1)
        pred = self.model(x)
        loss = self.criterion(pred, y)
        self.train_score(pred,y.to(torch.int))
        self.log("f1_score", self.train_score)
        self.log("loss", loss, prog_bar=True, logger=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x = x.view(x.size(0),12, -1)
        pred = self.model(x)
        val_loss = self.criterion(pred, y)
        self.val_score(pred,y.to(torch.int))
        self.log("val_f1_score", self.val_score)
        self.log("val_loss", val_loss)


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    


Если один батч - лосс converges в 0, если 5 - в пизде какой то. Если менять lr сильно ниче не меняется, если поменять sequence_len в препроцессинге(transforming.ipynb, BEAT_LENGTH) тоже особо не меняется. Когда sequence_len была 1400 и 5 батчей лосс конвержился в 19 хз.

In [10]:
class CWT_ResNet(L.LightningModule):
    def __init__(self, model_name, model_hparams, optimizer_name, optimizer_hparams):
        """
        Inputs:
            model_name - Name of the model/CNN to run. Used for creating the model (see function below)
            model_hparams - Hyperparameters for the model, as dictionary.
            optimizer_name - Name of the optimizer to use. Currently supported: Adam, SGD
            optimizer_hparams - Hyperparameters for the optimizer, as dictionary. This includes learning rate, weight decay, etc.
        """
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = ResNet(**model_hparams)
        # Create loss module
        self.loss_module = nn.BCEWithLogitsLoss()
        self.train_score = BinaryF1Score()
        self.val_score = BinaryF1Score()
        self.test_score = BinaryF1Score()
        self.val_acc = BinaryAccuracy()
        self.train_acc = BinaryAccuracy()
        # Example input for visualizing the graph in Tensorboard
        self.example_input_array = torch.zeros((1, 12, 32, 32), dtype=torch.float32)

    def forward(self, imgs):
        # Forward function that is run when visualizing the graph
        return self.model(imgs)

    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            # AdamW is Adam with a correct implementation of weight decay (see here
            # for details: https://arxiv.org/pdf/1711.05101.pdf)
            optimizer = optim.AdamW(self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(), **self.hparams.optimizer_hparams)
        else:
            assert False, f'Unknown optimizer: "{self.hparams.optimizer_name}"'

        # We will reduce the learning rate by 0.1 every milestone
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[65, 115, 150], gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        # "batch" is the output of the training data loader.
        imgs, labels = batch
        labels = np.squeeze(labels)
        preds = np.squeeze(self.model(imgs))
        loss = self.loss_module(preds, labels)
        self.train_acc(preds, np.squeeze(labels).to(torch.int))

        # Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.train_score(preds, labels.to(torch.int))
        self.log("train_f1_score", self.train_score)
        self.log("train_acc", self.train_acc, on_step=False, on_epoch=True)
        self.log("train_loss", loss)
        return loss  # Return tensor to call ".backward" on

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = np.squeeze(labels)
        preds = np.squeeze(self.model(imgs))
        self.val_acc(preds, labels.to(torch.int))
        # By default logs it per epoch (weighted average over batches)
        self.val_score(preds, labels.to(torch.int))
        self.log("val_f1_score", self.val_score)
        self.log("val_acc", self.val_acc)

    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        # By default logs it per epoch (weighted average over batches), and returns it afterwards
        self.test_score(preds, labels.to(torch.int))
        self.log("val_f1_score_test", self.test_score)
        self.log("test_acc", acc)

In [11]:
def train_model(model_name, save_name=None, **kwargs):
    """
    Inputs:
        model_name - Name of the model you want to run. Is used to look up the class in "model_dict"
        save_name (optional) - If specified, this name will be used for creating the checkpoint and logging directory.
    """
    if save_name is None:
        save_name = model_name

    # Create a PyTorch Lightning trainer with the generation callback
    trainer = L.Trainer(
        check_val_every_n_epoch=2,
        default_root_dir=os.path.join(CHECKPOINT_PATH, save_name),  # Where to save models
        # We run on a single GPU (if possible)
        accelerator="auto",
        devices=1,
        # How many epochs to train for if no patience is set
        max_epochs=180,
        callbacks=[
            ModelCheckpoint(
                mode="max", monitor="val_f1_score", save_top_k=2,
            ),  
            LearningRateMonitor("epoch"),
        ],  # Log learning rate every epoch
    )  # In case your notebook crashes due to the progress bar, consider increasing the refresh rate
    trainer.logger._log_graph = True  # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None  # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, save_name + ".ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        trainer.fit(model, train_loader, val_loader, ckpt_path=pretrained_filename)
        # Automatically loads the model with the saved hyperparameters
        model = CWT_ResNet.load_from_checkpoint(pretrained_filename)
    else:
        L.seed_everything(42)  # To be reproducable
        model = CWT_ResNet(model_name=model_name, **kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = CWT_ResNet.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )  # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    result = {"val": val_result[0]["test_acc"], "val_f1_score": val_result[0]["val_f1_score_test"]}

    return model, result

In [12]:
resnet_model, resnet_results = train_model(
    model_name="cwt_resNet_v3_lower_c_less_blocks",
    save_name="cwt_resNet_v4", 
    model_hparams={"num_classes": 1, "c_hidden": [16, 32, 64], "num_blocks": [3, 3, 3], "act_fn_name": "relu"},
    optimizer_name="SGD",
    optimizer_hparams={"lr": 0.0001, "momentum": 0.9, "weight_decay": 1e-4},
) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params | In sizes        | Out sizes
--------------------------------------------------------------------------------
0 | model       | ResNet            | 273 K  | [1, 12, 32, 32] | [1, 1]   
1 | loss_module | BCEWithLogitsLoss | 0      | ?               | ?        
2 | train_score | BinaryF1Score     | 0      | ?               | ?        
3 | val_score   | BinaryF1Score     | 0      | ?               | ?        
4 | test_score  | BinaryF1Score     | 0      | ?               | ?        
5 | val_acc     | BinaryAccuracy    | 0      | ?               | ?        
6 | train_acc   | BinaryAccuracy    | 0      | ?               | ?        
--------------------------------------------------------------------------------
273 K     Trainable params
0         Non-trainable params
273 K     Total params
1.092     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/mnt/sda5/conda/envs/open-mmlab/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

RuntimeError: Predictions and targets are expected to have the same shape, but got torch.Size([16]) and torch.Size([16, 1]).

In [ ]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

# model
autoencoder = LitBaseline(Baseline(sequence_len=768-128,n_classes=1,n_blocks=5))

checkpoint_callback =  ModelCheckpoint(dirpath="./lightning_logs/best_run1/", save_top_k=2, monitor="val_f1_score")
# train model
# trainer = pl.Trainer(max_epochs=100, check_val_every_n_epoch=5,enable_checkpointing=True,
#                      callbacks=[EarlyStopping(monitor="val_f1_score", mode='max', patience=15), checkpoint_callback])
# trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=val_loader,)

In [ ]:
trainer.validate(autoencoder, val_loader)

NameError: name 'trainer' is not defined

In [ ]:
overfit_logs = "./lightning_logs/version_52/metrics.csv"
df2 = pd.read_csv(overfit_logs)
df2

FileNotFoundError: [Errno 2] No such file or directory: './lightning_logs/version_52/metrics.csv'